<a href="https://colab.research.google.com/github/kabatyy/deepCaptioning/blob/main/dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import shutil
import subprocess


In [2]:
GIT_REPO_URL = "https://github.com/kabatyy/deepCaptioning.git"
COCO_API_REPO = "https://github.com/cocodataset/cocoapi.git"
DIR = "/content/project"
CLONE_DIR = os.path.join(DIR, "deepCaptioning")

def clone_repo():
    os.makedirs(DIR, exist_ok = True)
    os.chdir(DIR)
    if os.path.exists(CLONE_DIR):
        print(f"Repo already exists at {CLONE_DIR}, skipping clone.")
    else:
        subprocess.run(["git", "clone", GIT_REPO_URL], check=True)
        print(f"Repository cloned. Cloning COCO API...")
        os.chdir("/content/project/deepCaptioning")
        subprocess.run(["git", "clone", COCO_API_REPO], check=True)

def process_dataset():
  subprocess.run(["python3", 'start.py'], check=True)
  print("Dataset processing completed.")

def install_cocoapi():
  os.chdir("/content/project/deepCaptioning/cocoapi/PythonAPI")
  subprocess.run(["make"], check=True)
  print("COCO API installation completed.")

In [3]:
print("Cloning repo...")
clone_repo()

Cloning repo...
Repository cloned. cLONING COCO API...


In [4]:
print("Processing dataset...")
process_dataset()

Processing dataset...
Dataset processing completed.


In [5]:
print("Installing COCO API...")
install_cocoapi()


Installing COCO API...
COCO API installation completed.
